In [491]:
# import libraries

import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import librosa
import librosa.display

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
#from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict

RSEED=42



In [492]:
# read data
df=pd.read_csv('/Users/annelahann/neue-fische/RiffRefine/data/data_with_target.csv')

In [493]:
pd.crosstab(df['listens_bins'],df['track_license'],normalize='columns').T

listens_bins,1,2,3,4,5
track_license,,,,,
Art Libre,0.000000,0.000000,0.000000,0.500000,0.500000
Attribution,0.063985,0.146252,0.170018,0.325411,0.294333
Attribution 2.0 France,0.000000,0.000000,0.000000,0.000000,1.000000
Attribution 2.0 UK: England,0.000000,0.000000,0.000000,0.000000,1.000000
Attribution 2.5 Canada,0.000000,0.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...
Music Sharing,0.000000,0.000000,0.000000,1.000000,0.000000
Noncommercial Sampling Plus,0.000000,0.000000,0.125000,0.000000,0.875000
Public Domain,0.050000,0.375000,0.150000,0.250000,0.175000


In [494]:
df.groupby('listens_bins')['track_duration'].mean()

listens_bins
1    232.603081
2    231.606864
3    225.005255
4    228.928101
5    225.320513
Name: track_duration, dtype: float64

In [495]:
# Define predictors and target variable
mfcc = [col for col in df.columns if col.startswith('mfcc')]
chroma = [col for col in df.columns if col.startswith('chroma')]
rmse = [col for col in df.columns if col.startswith('rmse')]
zcr = [col for col in df.columns if col.startswith('zcr')]
spectral = [col for col in df.columns if col.startswith('spectral')]
tonnetz = [col for col in df.columns if col.startswith('tonnetz')]


X = df[['track_genre_top','track_bit_rate','track_duration','album_information_bin','artist_website_bin']+mfcc+chroma+rmse+zcr+spectral+tonnetz]
#X = df[['track_genre_top','track_bit_rate','track_duration','album_information_bin','artist_website_bin']]
#X=df[mfcc+['rmse_mean_01']]
y = df['listens_bins']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")



We have 8000 observations in our dataset and 523 features
Our target vector has also 8000 values


In [496]:
# Creating list for categorical predictors/features 

cat_features = list(X.columns[X.dtypes==object])
cat_features

['track_genre_top']

In [497]:
# Creating list for numerical predictors/features

num_features = list(X.columns[X.dtypes!=object])
num_features

['track_bit_rate',
 'track_duration',
 'album_information_bin',
 'artist_website_bin',
 'mfcc_kurtosis_01',
 'mfcc_kurtosis_02',
 'mfcc_kurtosis_03',
 'mfcc_kurtosis_04',
 'mfcc_kurtosis_05',
 'mfcc_kurtosis_06',
 'mfcc_kurtosis_07',
 'mfcc_kurtosis_08',
 'mfcc_kurtosis_09',
 'mfcc_kurtosis_10',
 'mfcc_kurtosis_11',
 'mfcc_kurtosis_12',
 'mfcc_kurtosis_13',
 'mfcc_kurtosis_14',
 'mfcc_kurtosis_15',
 'mfcc_kurtosis_16',
 'mfcc_kurtosis_17',
 'mfcc_kurtosis_18',
 'mfcc_kurtosis_19',
 'mfcc_kurtosis_20',
 'mfcc_max_01',
 'mfcc_max_02',
 'mfcc_max_03',
 'mfcc_max_04',
 'mfcc_max_05',
 'mfcc_max_06',
 'mfcc_max_07',
 'mfcc_max_08',
 'mfcc_max_09',
 'mfcc_max_10',
 'mfcc_max_11',
 'mfcc_max_12',
 'mfcc_max_13',
 'mfcc_max_14',
 'mfcc_max_15',
 'mfcc_max_16',
 'mfcc_max_17',
 'mfcc_max_18',
 'mfcc_max_19',
 'mfcc_max_20',
 'mfcc_mean_01',
 'mfcc_mean_02',
 'mfcc_mean_03',
 'mfcc_mean_04',
 'mfcc_mean_05',
 'mfcc_mean_06',
 'mfcc_mean_07',
 'mfcc_mean_08',
 'mfcc_mean_09',
 'mfcc_mean_10',
 'm

In [498]:
# Split into train and test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

In [499]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (6400, 523)
X_test shape: (1600, 523)
y_train shape: (6400,)
y_test shape: (1600,)


In [500]:
from sklearn.pipeline import Pipeline

# Pipeline for numerical features

num_pipeline = Pipeline([
    ('imputer_num', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

# Pipeline for categorical features 

cat_pipeline = Pipeline([
    ('imputer_cat', SimpleImputer(strategy='constant', fill_value='missing')),
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])

In [501]:
from sklearn.compose import ColumnTransformer

# Complete pipeline for numerical and categorical features
# 'ColumnTransformer' applies transformers (num_pipeline/ cat_pipeline) to specific columns of a DataFrame 

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

In [502]:
# Building a full pipeline with our preprocessor and a KNN Classifier

pipe_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

In [503]:
# Building a full pipeline with our preprocessor and a DT Classifier

pipe_dtc = Pipeline([
    ('preprocessor', preprocessor),
    ('dtc', DecisionTreeClassifier(max_depth=10,min_samples_leaf=20,max_features=None))
])

In [504]:
# Making predictions on the training and test set using cross validation 

y_train_predicted = cross_val_predict(pipe_knn, X_train, y_train, cv=5)
y_test_predicted = cross_val_predict(pipe_knn, X_test, y_test, cv=5)

In [505]:
# Making predictions on the training and test set using cross validation 

y_train_pred_dtc = cross_val_predict(pipe_dtc, X_train, y_train, cv=5)
y_test_pred_dtc = cross_val_predict(pipe_dtc, X_test, y_test, cv=5)

In [506]:
# Performance KNN Train Data

conf_matrix= confusion_matrix(y_train, y_train_predicted)
accurate_classes = sum([conf_matrix[i][i] for i in range(len(conf_matrix))])
off_by_one = sum([conf_matrix[i][j] if abs(i - j) == 1 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_two = sum([conf_matrix[i][j] if abs(i - j) == 2 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_three = sum([conf_matrix[i][j] if abs(i - j) == 3 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_four = sum([conf_matrix[i][j] if abs(i - j) == 4 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])

print((100*accurate_classes/conf_matrix.sum()).round(1))
print((100*off_by_one/conf_matrix.sum()).round(1))
print((100*off_by_two/conf_matrix.sum()).round(1))
print((100*off_by_three/conf_matrix.sum()).round(1))
print((100*off_by_four/conf_matrix.sum()).round(1))
print('--------------------------------')
print(cohen_kappa_score(y_train, y_train_predicted).round(3))

28.8
35.4
19.6
12.5
3.8
--------------------------------
0.102


In [507]:
# Performance KNN Test Data

conf_matrix= confusion_matrix(y_test, y_test_predicted)
accurate_classes = sum([conf_matrix[i][i] for i in range(len(conf_matrix))])
off_by_one = sum([conf_matrix[i][j] if abs(i - j) == 1 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_two = sum([conf_matrix[i][j] if abs(i - j) == 2 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_three = sum([conf_matrix[i][j] if abs(i - j) == 3 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_four = sum([conf_matrix[i][j] if abs(i - j) == 4 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])

print((100*accurate_classes/conf_matrix.sum()).round(1))
print((100*off_by_one/conf_matrix.sum()).round(1))
print((100*off_by_two/conf_matrix.sum()).round(1))
print((100*off_by_three/conf_matrix.sum()).round(1))
print((100*off_by_four/conf_matrix.sum()).round(1))
print('--------------------------------')
print(cohen_kappa_score(y_test, y_test_predicted).round(3))

27.7
36.1
18.3
13.9
4.1
--------------------------------
0.084


In [508]:
# Performance DT Train Data

conf_matrix= confusion_matrix(y_train, y_train_pred_dtc)
accurate_classes = sum([conf_matrix[i][i] for i in range(len(conf_matrix))])
off_by_one = sum([conf_matrix[i][j] if abs(i - j) == 1 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_two = sum([conf_matrix[i][j] if abs(i - j) == 2 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_three = sum([conf_matrix[i][j] if abs(i - j) == 3 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_four = sum([conf_matrix[i][j] if abs(i - j) == 4 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])

print((100*accurate_classes/conf_matrix.sum()).round(1))
print((100*off_by_one/conf_matrix.sum()).round(1))
print((100*off_by_two/conf_matrix.sum()).round(1))
print((100*off_by_three/conf_matrix.sum()).round(1))
print((100*off_by_four/conf_matrix.sum()).round(1))
print('--------------------------------')
print(cohen_kappa_score(y_train, y_train_pred_dtc).round(3))

39.3
40.3
13.2
5.8
1.3
--------------------------------
0.235


In [509]:
# Performance DT Test Data

conf_matrix= confusion_matrix(y_test, y_test_pred_dtc)
accurate_classes = sum([conf_matrix[i][i] for i in range(len(conf_matrix))])
off_by_one = sum([conf_matrix[i][j] if abs(i - j) == 1 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_two = sum([conf_matrix[i][j] if abs(i - j) == 2 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_three = sum([conf_matrix[i][j] if abs(i - j) == 3 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])
off_by_four = sum([conf_matrix[i][j] if abs(i - j) == 4 else 0 for i in range(len(conf_matrix)) for j in range(len(conf_matrix))])

print((100*accurate_classes/conf_matrix.sum()).round(1))
print((100*off_by_one/conf_matrix.sum()).round(1))
print((100*off_by_two/conf_matrix.sum()).round(1))
print((100*off_by_three/conf_matrix.sum()).round(1))
print((100*off_by_four/conf_matrix.sum()).round(1))
print('--------------------------------')
print(cohen_kappa_score(y_test, y_test_pred_dtc).round(3))

38.7
40.1
13.0
6.8
1.4
--------------------------------
0.227


In [510]:
# Defining parameter space for grid-search for decision trees

param_dtc = {   'max_depth': [None, 5, 10, 20, 30],
                'min_samples_split': [5,10,20],
                'min_samples_leaf': [2,5,10],
                'max_features':['auto', 'sqrt', 'log2']
               }

grid_dtc = GridSearchCV(pipe_dtc, param_grid=param_dtc, cv=5, scoring= cohen_kappa_score, 
                           verbose=5, n_jobs=-1)



In [511]:
grid_dtc.fit(X_train, y_train)

Fitting 5 folds for each of 135 candidates, totalling 675 fits


ValueError: Invalid parameter 'max_depth' for estimator Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['track_bit_rate',
                                                   'track_duration',
                                                   'album_information_bin',
                                                   'artist_website_bin',
                                                   'mfcc_kurtosis_01',
                                                   'mfcc_kurtosis_02',
                                                   'mfcc_kurtosis_03',
 